In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import requests
import json
import re
import geopy.geocoders as gc
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
tqdm.pandas()

In [2]:
decisions_data = pd.read_csv("data/Filtered_ZBA_decisions.csv")
meetings_data = pd.read_csv("data/Filtered_ZBA_meetings.csv")
decisions_data

,Source,Case Number,Applicant Address,Decision (Approve/Reject)
0,ZBA Decision 2017,BOA-639633,"105 Allston Street, Ward 21",Approved
1,ZBA Decision 2017,BOA-652473,"65 Ashland Street, Ward 16",Approved
2,ZBA Decision 2017,BOA-640152,"29 Batchelder Street, Ward 8",Approved
3,ZBA Decision 2017,BOA-640141,"21-27 Batchelder Street, Ward 8",Approved
4,ZBA Decision 2017,BOA-568164,"474 Beacon Street, Ward 5",Approved
...,...,...,...,...
2916,ZBA Decision 2019,BOA-892970,"91A Baker Street, Ward 20",Denied
2917,ZBA Decision 2019,BOA-822030,"212 Chelsea Street, Ward 1",Denied
2918,ZBA Decision 2019,BOA-864614,"118-120 Granite Avenue, Ward 16",Denied
2919,ZBA Decision 2019,BOA-865287,"54 High Street, Ward 2",Denied


In [3]:
meetings_data

,Source,Case Number,Applicant,Applicant Address
0,ZBA Meeting 2018,BOA-379892,Michael Cronin,42 Thompson Street Ward 18
1,ZBA Meeting 2018,BOA-645234,Andrew Kara,45-55 Brighton Avenue Ward 21
2,ZBA Meeting 2018,BOA-645231,Andrew Kara,75 Gardner Street Ward 21
3,ZBA Meeting 2018,BOA-382897,Ruth Silman,104 Canal Street Ward 3
4,ZBA Meeting 2018,BOA-694026,Richard Lynds,77-85 Liverpool Street Ward 1
...,...,...,...,...
2998,ZBA Meeting 2017,BOA719303,James Christopher,"435 Hyde Park AV, Ward - 19"
2999,ZBA Meeting 2017,BOA730816,Elba Rojas,"82 Cummins HW, Ward - 19"
3000,ZBA Meeting 2017,BOA730716,John Harrington,"177 Glenellen Rd, Ward - 20"
3001,ZBA Meeting 2017,BOA732458,Patrick Mahoney,"118 - 122 Tremont St, Ward - 22"


In [4]:
decisions_data['Case Number'] = decisions_data.apply(lambda row: re.sub("[^0-9]", "", row['Case Number']), axis=1)
decisions_data

,Source,Case Number,Applicant Address,Decision (Approve/Reject)
0,ZBA Decision 2017,639633,"105 Allston Street, Ward 21",Approved
1,ZBA Decision 2017,652473,"65 Ashland Street, Ward 16",Approved
2,ZBA Decision 2017,640152,"29 Batchelder Street, Ward 8",Approved
3,ZBA Decision 2017,640141,"21-27 Batchelder Street, Ward 8",Approved
4,ZBA Decision 2017,568164,"474 Beacon Street, Ward 5",Approved
...,...,...,...,...
2916,ZBA Decision 2019,892970,"91A Baker Street, Ward 20",Denied
2917,ZBA Decision 2019,822030,"212 Chelsea Street, Ward 1",Denied
2918,ZBA Decision 2019,864614,"118-120 Granite Avenue, Ward 16",Denied
2919,ZBA Decision 2019,865287,"54 High Street, Ward 2",Denied


In [5]:
meetings_data = meetings_data[meetings_data['Case Number'].notna()]
meetings_data['Case Number'] = meetings_data.apply(lambda row: re.sub("[^0-9]", "", row['Case Number']), axis=1)
meetings_data

,Source,Case Number,Applicant,Applicant Address
0,ZBA Meeting 2018,379892,Michael Cronin,42 Thompson Street Ward 18
1,ZBA Meeting 2018,645234,Andrew Kara,45-55 Brighton Avenue Ward 21
2,ZBA Meeting 2018,645231,Andrew Kara,75 Gardner Street Ward 21
3,ZBA Meeting 2018,382897,Ruth Silman,104 Canal Street Ward 3
4,ZBA Meeting 2018,694026,Richard Lynds,77-85 Liverpool Street Ward 1
...,...,...,...,...
2998,ZBA Meeting 2017,719303,James Christopher,"435 Hyde Park AV, Ward - 19"
2999,ZBA Meeting 2017,730816,Elba Rojas,"82 Cummins HW, Ward - 19"
3000,ZBA Meeting 2017,730716,John Harrington,"177 Glenellen Rd, Ward - 20"
3001,ZBA Meeting 2017,732458,Patrick Mahoney,"118 - 122 Tremont St, Ward - 22"


In [6]:
meetings_with_decisions = pd.merge(meetings_data, decisions_data, left_on="Case Number", right_on="Case Number")
meetings_with_decisions = meetings_with_decisions.drop(columns=['Applicant Address_x', 'Source_x', 'Source_y'])
meetings_with_decisions = meetings_with_decisions.rename(columns={'Applicant Address_y': 'Address'})
meetings_with_decisions

,Case Number,Applicant,Address,Decision (Approve/Reject)
0,645234,Andrew Kara,"45-55 Brighton Avenue, Ward 21",Approved
1,694026,Richard Lynds,"86-86A Chestnut Street, Ward 41",Approved
2,896003,Andreas Hwang,"211 West Springfield Street, Ward 4",Approved
3,897604,Adam Grassi Article(s),"16 Marlborough Street, Ward 5",Approved
4,818377,Michael P. Ross,"111 Terrace Street, Ward 10",Approved
...,...,...,...,...
2423,733438,Joe Grealish,"26 Browning Avenue, Ward 14",Approved
2424,733442,Joe Grealish,"24 Browning Avenue, Ward 14",Approved
2425,719303,James Christopher,"435 Hyde Park Avenue, Ward 19",Denied
2426,730816,Elba Rojas,"82 Cummins Highway, Ward 19",Approved


In [7]:
gc.options.default_timeout = 10
geolocator = gc.Nominatim(user_agent="Baystate_Banner_Boston_Building_Binvestigators")
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.05)
location = geolocator.geocode("175 5th Avenue NYC")
location.latitude

43.3545384

In [11]:
meetings_with_decisions['geocodes'] = meetings_with_decisions['Address'].progress_apply(geolocator.geocode)
meetings_with_decisions

100%|██████████| 2428/2428 [22:31<00:00,  1.80it/s]


,Case Number,Applicant,Address,Decision (Approve/Reject),geocodes
0,645234,Andrew Kara,"45-55 Brighton Avenue, Ward 21",Approved,"(Brighton Avenue, Ekurhuleni Ward 21, City of ..."
1,694026,Richard Lynds,"86-86A Chestnut Street, Ward 41",Approved,"(86, Chestnut Street, Ward 1, Newport, Newport..."
2,896003,Andreas Hwang,"211 West Springfield Street, Ward 4",Approved,"(Springfield Street, Cape Town Ward 43, Cape T..."
3,897604,Adam Grassi Article(s),"16 Marlborough Street, Ward 5",Approved,"(16, Marlborough Street, Ward 2, Newport, Newp..."
4,818377,Michael P. Ross,"111 Terrace Street, Ward 10",Approved,"(store, Tagbilaran East Road, Purok 5, Canlang..."
...,...,...,...,...,...
2423,733438,Joe Grealish,"26 Browning Avenue, Ward 14",Approved,"(26, Browning Avenue, Gateway Hills, Ward 8, N..."
2424,733442,Joe Grealish,"24 Browning Avenue, Ward 14",Approved,"(24, Browning Avenue, Gateway Hills, Ward 8, N..."
2425,719303,James Christopher,"435 Hyde Park Avenue, Ward 19",Denied,"(Hyde Park Avenue, Cape Town Ward 69, Noordhoe..."
2426,730816,Elba Rojas,"82 Cummins Highway, Ward 19",Approved,None


In [14]:
filtered_meetings_with_decisions = meetings_with_decisions[meetings_with_decisions['geocodes'].notna()]
filtered_meetings_with_decisions

,Case Number,Applicant,Address,Decision (Approve/Reject),geocodes
0,645234,Andrew Kara,"45-55 Brighton Avenue, Ward 21",Approved,"(Brighton Avenue, Ekurhuleni Ward 21, City of ..."
1,694026,Richard Lynds,"86-86A Chestnut Street, Ward 41",Approved,"(86, Chestnut Street, Ward 1, Newport, Newport..."
2,896003,Andreas Hwang,"211 West Springfield Street, Ward 4",Approved,"(Springfield Street, Cape Town Ward 43, Cape T..."
3,897604,Adam Grassi Article(s),"16 Marlborough Street, Ward 5",Approved,"(16, Marlborough Street, Ward 2, Newport, Newp..."
4,818377,Michael P. Ross,"111 Terrace Street, Ward 10",Approved,"(store, Tagbilaran East Road, Purok 5, Canlang..."
...,...,...,...,...,...
2421,726332,Kim Dung Nguyen,"110 Moore Street, Ward 33",Approved,"(110, Moore Street, Sweetwater Estates, Ward, ..."
2422,733431,Joe Grealish,"28 Browning Avenue, Ward 14",Approved,"(28, Browning Avenue, Gateway Hills, Ward 8, N..."
2423,733438,Joe Grealish,"26 Browning Avenue, Ward 14",Approved,"(26, Browning Avenue, Gateway Hills, Ward 8, N..."
2424,733442,Joe Grealish,"24 Browning Avenue, Ward 14",Approved,"(24, Browning Avenue, Gateway Hills, Ward 8, N..."


In [15]:
filtered_meetings_with_decisions['lat'] = filtered_meetings_with_decisions.progress_apply(lambda row: row['geocodes'].latitude, axis=1)
filtered_meetings_with_decisions['long'] = filtered_meetings_with_decisions.progress_apply(lambda row: row['geocodes'].longitude, axis=1)
filtered_meetings_with_decisions

100%|██████████| 1391/1391 [00:00<00:00, 115899.74it/s]


,Case Number,Applicant,Address,Decision (Approve/Reject),geocodes,lat,long
0,645234,Andrew Kara,"45-55 Brighton Avenue, Ward 21",Approved,"(Brighton Avenue, Ekurhuleni Ward 21, City of ...",-26.183166,28.159704
1,694026,Richard Lynds,"86-86A Chestnut Street, Ward 41",Approved,"(86, Chestnut Street, Ward 1, Newport, Newport...",41.495169,-71.320874
2,896003,Andreas Hwang,"211 West Springfield Street, Ward 4",Approved,"(Springfield Street, Cape Town Ward 43, Cape T...",-34.001157,18.527376
3,897604,Adam Grassi Article(s),"16 Marlborough Street, Ward 5",Approved,"(16, Marlborough Street, Ward 2, Newport, Newp...",41.491076,-71.313965
4,818377,Michael P. Ross,"111 Terrace Street, Ward 10",Approved,"(store, Tagbilaran East Road, Purok 5, Canlang...",9.710126,124.406939
...,...,...,...,...,...,...,...
2421,726332,Kim Dung Nguyen,"110 Moore Street, Ward 33",Approved,"(110, Moore Street, Sweetwater Estates, Ward, ...",35.029328,-91.949508
2422,733431,Joe Grealish,"28 Browning Avenue, Ward 14",Approved,"(28, Browning Avenue, Gateway Hills, Ward 8, N...",42.719263,-71.470885
2423,733438,Joe Grealish,"26 Browning Avenue, Ward 14",Approved,"(26, Browning Avenue, Gateway Hills, Ward 8, N...",42.719241,-71.470979
2424,733442,Joe Grealish,"24 Browning Avenue, Ward 14",Approved,"(24, Browning Avenue, Gateway Hills, Ward 8, N...",42.719218,-71.471073


In [16]:
sub_neigh = pd.read_csv("data/Sub-Neighborhoods.csv")
sub_neigh['Census Tract#'] = sub_neigh.progress_apply(lambda row: float(row['Census Tract#']), axis=1)
sub_neigh['White Proportion'] = sub_neigh.progress_apply(lambda row: float(row['White Proportion'][:-1]), axis=1)
sub_neigh

100%|██████████| 181/181 [00:00<00:00, 90474.20it/s]


,Census Tract#,Zipcode,Neighborhood,US Geonames Sub-neigborhood,Total Population,Black,Black Proportion,White,White Proportion,American Indian & Alaska Native,...,Male,Male Proportion,Female,Female Proportion,Lat,Long,Primary Sub-Neighborhood,Secondary Sub-Neighborhood,Teritriary Sub-Neighborhood(s),T-Stop/Commuter Rail
0,1.00,"02134, 02135",Brighton,NaN,"5,324",311,5.84%,"3,315",62.27,0,...,"2,578",48.42%,"2,746",51.58%,42.361484,-71.138588,Lower Allston,Allston,Packard's Corner,Boston Landing
1,2.01,02135,Brighton,NaN,"3,991",237,5.94%,"3,164",79.28,17,...,"1,800",45.10%,"2,191",54.90%,42.354066,-71.161168,Oak Square,Brighton,Hunnewell Hill,NaN
2,2.02,02135,Brighton,NaN,"4,272",263,6.16%,"3,031",70.95,0,...,"2,175",50.91%,"2,097",49.09%,42.352605,-71.154344,Oak Square,Brighton,Allston,NaN
3,3.01,02135,Brighton,NaN,"2,946",216,7.33%,"2,203",74.78,0,...,"1,650",56.01%,"1,296",43.99%,42.354144,-71.168827,Hunnwell Hill,Oak Square,NaN,NaN
4,3.02,02135,Brighton,NaN,"3,469",359,10.35%,"2,346",67.63,25,...,"1,733",49.96%,"1,736",50.04%,42.347258,-71.167686,Brighton,St. Elizabeth's,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,9815.02,02151,East Boston,NaN,0,0,0.00%,0,0.00,0,...,0,0.00%,0,0.00%,42.396367,-71.004379,Beachmont,Orient Heights,NaN,Beachmont
177,9816.00,02128,East Boston,NaN,0,0,0.00%,0,0.00,0,...,0,0.00%,0,0.00%,42.388621,-70.993442,Orient Heights,Beachmont,NaN,Suffolk Downs
178,9817.00,02116,Beacon Hill,NaN,0,0,0.00%,0,0.00,0,...,0,0.00%,0,0.00%,42.355087,-71.065743,Downtown Crossing,NaN,NaN,Boylston Street
179,9818.00,"02130, 02215",Jamaica Plain,NaN,26,9,34.62%,17,65.38,0,...,4,15.38%,22,84.62%,42.323121,-71.116882,High Street Hill,NaN,NaN,Heath Street / Back of the Hill


In [28]:
# Function to convert the Lat and Long into a census tract.
# Refer to https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html for the FIPS code to census tract conversion
# FCC Census block conversion API link: https://geo.fcc.gov/api/census/#!/block/get_block_find

def Coord_to_census_tract(Lat,Long):
    Converter = requests.get("https://geo.fcc.gov/api/census/block/find?latitude=%20" + str(Lat) + "&longitude=" + str(Long) + "&showall=true&format=json")
    #print(Converter.text)
    Census_data = Converter.text
    json_data = json.loads(Census_data)
    # print(json_data)
    FIPS_code = json_data['Block']['FIPS']
    if FIPS_code is None:
        return "nan"
    Census_tract = FIPS_code[5:11]
    #print(type(Census_tract))
    if Census_tract[0] == "0":
        Census_tract.replace(Census_tract[0],'')
        Census_split = Census_tract[ : 4] + '.'+ Census_tract[4 : ]
        return(Census_split)
    else:
        Census_split = Census_tract[ : 4] + '.'+ Census_tract[4 : ]
        return(Census_split)

In [30]:
filtered_meetings_with_decisions['Cencus Tract'] = filtered_meetings_with_decisions.progress_apply(lambda row: float(Coord_to_census_tract(row.lat, row.long)), axis=1)
filtered_meetings_with_decisions = filtered_meetings_with_decisions[filtered_meetings_with_decisions['Cencus Tract'].notna()]
decisions_with_demos = pd.merge(filtered_meetings_with_decisions, sub_neigh, left_on="Cencus Tract", right_on="Census Tract#")
decisions_with_demos

100%|██████████| 1391/1391 [02:44<00:00,  8.44it/s]


,Case Number,Applicant,Address,Decision (Approve/Reject),geocodes,lat,long,Cencus Tract,Census Tract#,Zipcode,...,Male,Male Proportion,Female,Female Proportion,Lat,Long,Primary Sub-Neighborhood,Secondary Sub-Neighborhood,Teritriary Sub-Neighborhood(s),T-Stop/Commuter Rail
0,893682,37-43 North Beacon Street LLC,"37 North Beacon Street, Ward 22",Approved,"(37, Beacon Street, Nashville Historic Distric...",42.765158,-71.470898,105.0,105.0,"02115, 02116",...,"1,907",54.15%,"1,615",45.85%,42.344564,-71.084575,Back Bay,Columbus,Frederick Douglas Square Historic District,Massachusetts Ave
1,893682,37-43 North Beacon Street LLC,"37 North Beacon Street, Ward 22",Approved,"(37, Beacon Street, Nashville Historic Distric...",42.765158,-71.470898,105.0,105.0,"02115, 02116",...,"1,907",54.15%,"1,615",45.85%,42.344564,-71.084575,Back Bay,Columbus,Frederick Douglas Square Historic District,Massachusetts Ave
2,845481,Frank Scire,"64-66 Salem Street, Ward 3",Denied,"(64, Salem Street, Nashville Historic District...",42.767406,-71.460873,105.0,105.0,"02115, 02116",...,"1,907",54.15%,"1,615",45.85%,42.344564,-71.084575,Back Bay,Columbus,Frederick Douglas Square Historic District,Massachusetts Ave
3,845481,Frank Scire,"64-66 Salem Street, Ward 3",Denied,"(64, Salem Street, Nashville Historic District...",42.767406,-71.460873,105.0,105.0,"02115, 02116",...,"1,907",54.15%,"1,615",45.85%,42.344564,-71.084575,Back Bay,Columbus,Frederick Douglas Square Historic District,Massachusetts Ave
4,845481,Frank Scire,"64-66 Salem Street, Ward 3",Denied,"(64, Salem Street, Nashville Historic District...",42.767406,-71.460873,105.0,105.0,"02115, 02116",...,"1,907",54.15%,"1,615",45.85%,42.344564,-71.084575,Back Bay,Columbus,Frederick Douglas Square Historic District,Massachusetts Ave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,871519,Russell Flynne,"218 Everett Street, Ward 1",Approved,"(218, Everett Street, Manchester, Richmond, Ri...",37.523271,-77.433732,610.0,610.0,02127,...,"1,483",45.96%,"1,744",54.04%,42.331621,-71.049917,Columbus Park/Andrew Square,Telegraph Hill,NaN,E 8th St @ Mercer St
80,687529,"Richard Lynds, Esq","254 Everett Street, Ward 1",Approved,"(254, Everett Street, Manchester, Richmond, Ri...",37.523140,-77.433936,610.0,610.0,02127,...,"1,483",45.96%,"1,744",54.04%,42.331621,-71.049917,Columbus Park/Andrew Square,Telegraph Hill,NaN,E 8th St @ Mercer St
81,687529,Richard Lynds,"254 Everett Street, Ward 1",Approved,"(254, Everett Street, Manchester, Richmond, Ri...",37.523140,-77.433936,610.0,610.0,02127,...,"1,483",45.96%,"1,744",54.04%,42.331621,-71.049917,Columbus Park/Andrew Square,Telegraph Hill,NaN,E 8th St @ Mercer St
82,740260,Tuan Nguyen,"411 Adams Street, Ward 16",Approved,"(411, Adams Street, Ward Lee Mobile Home Park,...",43.694813,-91.276732,202.0,202.0,02114,...,"1,800",48.77%,"1,891",51.23%,42.360335,-71.068633,West End,Bay Village,NaN,Charles/MGH


In [ ]:
decisions_with_demos

In [ ]:
grouped_neighs = decisions_with_demos.groupby('Neighborhood')
grouped_neighs_approved_size = grouped_neighs.filter(lambda row: row['Decision (Approve/Reject)'] == 'Approved').agg("size")
grouped_neighs_approved_size

In [ ]:
grouped_neighs_approved_size/grouped_neighs.agg("size")